In [ ]:
#성공한 리스트 링크의 feature수집

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import operator
import numpy as np

driver = webdriver.Chrome('C:/Users/tlath/Desktop/chromedriver')

In [4]:
number = [1,2,3,4,5,6,7,8,9,10]   #세부 사이트 들어가는 링크 크롤링
success_link_list = list()

for i in number:
    url="https://www.kickstarter.com/discover/advanced?state=successful&woe_id=0&sort=magic&seed=2653556&page="+str(i)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    a_tags = soup.find_all("a", {'class':"block img-placeholder w100p"}) 
    
    for line in a_tags :
        title=line["href"]
        success_link_list.append(title)
        
print(success_link_list)

['https://www.kickstarter.com/projects/sjgames/car-wars-sixth-edition-by-steve-jackson-games?ref=discovery', 'https://www.kickstarter.com/projects/transolargames/hero-u-rogue-to-redemption?ref=discovery', 'https://www.kickstarter.com/projects/68voces/68-voces-68-corazones-68-voices-68-hearts?ref=discovery', 'https://www.kickstarter.com/projects/nuchallenger/treachery-in-beatdown-city-a-tactical-beat-em-up-0?ref=discovery', 'https://www.kickstarter.com/projects/leonardo-corbucci/legendary-ad?ref=discovery', 'https://www.kickstarter.com/projects/mouthcongress/mouth-congress?ref=discovery', 'https://www.kickstarter.com/projects/167825245/alders-blood-a-victorian-western-tactical-stealth-game?ref=discovery', 'https://www.kickstarter.com/projects/ninabolenart/happy-dragons-whimsical-dragon-sculptures-by-nina?ref=discovery', 'https://www.kickstarter.com/projects/starsoft/savior-4?ref=discovery', 'https://www.kickstarter.com/projects/failbetter/sunless-sea?ref=discovery', 'https://www.kicksta

In [5]:
money_list=[]
pledged_money_list=[]
creator_list=[]
backers_list=[]
final_created_list=[]
city_list=[]
category_list=[]
support_bankroll_list=[]
support_max_money_list=[]
support_min_money_list=[]
support_max_backers_list=[]
support_min_backers_list=[]
level_num_list=[]
support_backers_list=[]

In [6]:
for i in range(len(success_link_list)):
    url = success_link_list[i]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    print("링크",url)

    #실제로모인금액
    money = soup.select('span.money')[0].text
    money = re.findall('\d+',money)
    money = "".join(money)
    money_list.append(money)
    print("모인금액",money)

    #목표금액
    pledged_money = soup.select('span.money')[2].text
    pledged_money = re.findall('\d+',pledged_money)
    pledged_money = "".join(pledged_money)   
    pledged_money_list.append(pledged_money)
    print("목표금액",pledged_money)

    #만든사람
    creator = soup.select('a.hero__link')[1].text
    creator = creator.strip()
    creator_list.append(creator)
    print("만든사람",creator)

    #후원자수
    backers = soup.select('h3.mb0')[1].text
    backers = backers.strip()
    backers_list.append(backers)
    print("총 후원자수",backers)

    #제작자의프로젝트개설횟수
    a_tags = soup.find_all("a", {'class':"hero__link remote_modal_dialog js-update-text-color"}) 
    
    for created in a_tags :
        print(created["href"]) # 찾은 a태그의 href 값 크롤링
    url = "https://www.kickstarter.com" + created["href"]
    driver.get(url)     
    html_s = driver.page_source
    soup_s = BeautifulSoup(html_s, 'html.parser')

    created_dummy = driver.find_element_by_class_name("created-projects.py2.f5.mb3")
    final_created = re.findall('\d+',created_dummy.text)

    for i in range(len(final_created)):
        final_created[i] = int(final_created[i])
    if len(final_created) == 1:
        created_num = 0
    else:
        created_num = final_created[0]
    final_created_list.append(created_num)
    
    print("개설횟수",final_created_list)

    #제작자지역
    city = soup.select('a.grey-dark')[4].text
    city = city.strip()
    city_list.append(city)
    print("제작자지역",city)

    #프로젝트장르
    category = soup.select('a.grey-dark')[5].text
    category = category.strip()
    category_list.append(category)
    print("장르",category)

    #서포트금액
    support_money = soup.select('span.money')
    support_bankroll = []
    for line in support_money:
        line = line.get_text()         
        line = re.findall('\d+',line)
        line = "".join(line)
        support_bankroll.append(int(line))       
    for i in range(3):
        support_bankroll.pop(0)

    support_bankroll_list.append(support_bankroll)
    print("서포트금액",support_bankroll)

    #서포트금액 최소/최대
    support_max_idx = 0
    support_min_idx = 0
    max = support_bankroll[0]
    min = support_bankroll[0]
    
    for i in range(len(support_bankroll)):  
        if max < support_bankroll[i]:
            max = support_bankroll[i]
            support_max_idx = i
        if min > support_bankroll[i]:
            min = support_bankroll[i]
            support_min_idx = i
    print("최대금액",max)
    print("최소금액",min)
    support_max_money_list.append(max)
    support_min_money_list.append(min)

    #단계개수
    level_num = len(support_bankroll)
    level_num_list.append(level_num)
    print("서포트 단계",level_num)

    #서포트후원자수
    support_backer = soup.select(' div > div > div > div > div > div > div > ol > li > div > div > span')
    support_backers = []
    for line in support_backer:
        line = line.get_text() 
        if line == 'Includes:':
            continue
        line = line.replace('\n','')
        if line == 'Limited':
            continue
        if line == 'Reward no longer available':
            continue
        line = line.replace(' backers','')
        line = line.replace(' backer','')
        support_backers.append(line)
    support_backers_list.append(support_backers)
    print("서포트 금액별 후원자수",support_backers)

    #서포트최소/최대후원자수
    a_ar = np.array(support_bankroll)
    max_money_index = np.argmax(support_bankroll)
    min_money_index = np.argmin(support_bankroll)
    max_money_backers = support_backers[max_money_index]
    min_money_backers = support_backers[min_money_index]
    support_max_backers_list.append(max_money_backers)
    support_min_backers_list.append(min_money_backers)
    print("최대 금액 후원자수",max_money_backers)
    print("최소 금액 후원자수",min_money_backers)
    

링크 https://www.kickstarter.com/projects/sjgames/car-wars-sixth-edition-by-steve-jackson-games?ref=discovery
모인금액 652995
목표금액 80000
만든사람 Steve Jackson Games
총 후원자수 3,936
/projects/sjgames/car-wars-sixth-edition-by-steve-jackson-games/creator_bio
개설횟수 [25]
제작자지역 Austin, TX
장르 Tabletop Games
서포트금액 [51, 75, 100, 140, 200, 300, 1000, 1250, 50, 74, 139, 145, 146, 400]
최대금액 1250
최소금액 50
서포트 단계 14
서포트 금액별 후원자수 ['24', '312', '2', '650', '54', '11', '1', '4', '9', '206', '1,451', '542', '459', '90']
최대 금액 후원자수 4
최소 금액 후원자수 9
링크 https://www.kickstarter.com/projects/transolargames/hero-u-rogue-to-redemption?ref=discovery
모인금액 409150
목표금액 400000
만든사람 Corey Cole
총 후원자수 6,093
/projects/transolargames/hero-u-rogue-to-redemption/creator_bio
개설횟수 [25, 3]
제작자지역 San Francisco, CA
장르 Video Games
서포트금액 [5, 20, 35, 50, 65, 75, 100, 125, 150, 175, 200, 250, 500, 1050, 2500, 5000, 7500, 1000, 2000, 2500]
최대금액 7500
최소금액 5
서포트 단계 20
서포트 금액별 후원자수 ['69', '3,299', '893', '599', '156', '284', '145', '98', '186', '57

개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8]
제작자지역 Seattle, WA
장르 Letterpress
서포트금액 [10, 100, 200, 1000, 900]
최대금액 1000
최소금액 10
서포트 단계 5
서포트 금액별 후원자수 ['260', '18', '38', '49', '10']
최대 금액 후원자수 49
최소 금액 후원자수 260
링크 https://www.kickstarter.com/projects/1176616619/shadows-of-esteren-a-medieval-horror-rpg-travels?ref=discovery
모인금액 137024
목표금액 15000
만든사람 Jim Searcy
총 후원자수 951
/projects/1176616619/shadows-of-esteren-a-medieval-horror-rpg-travels/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9]
제작자지역 Indianapolis, IN
장르 Tabletop Games
서포트금액 [15, 40, 40, 70, 105, 110, 210, 230, 250, 320, 400, 3000, 90, 100, 125, 210, 325, 330]
최대금액 3000
최소금액 15
서포트 단계 18
서포트 금액별 후원자수 ['65', '45', '162', '53', '15', '243', '39', '44', '10', '69', '3', '0', '100', '50', '12', '1', '9', '4']
최대 금액 후원자수 0
최소 금액 후원자수 65
링크 https://www.kickstarter.com/projects/353596452/kefirko-home-made-probiotic?ref=discovery
모인금액 52694
목표금액 30000
만든사람 Marko Borko and Andrej Glažar
총 후원자수 1,137
/p

개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7]
제작자지역 St. Paul, MN
장르 Tabletop Games
서포트금액 [90]
최대금액 90
최소금액 90
서포트 단계 1
서포트 금액별 후원자수 ['12,299']
최대 금액 후원자수 12,299
최소 금액 후원자수 12,299
링크 https://www.kickstarter.com/projects/730652731/foreign-puzzle-a-film-about-love-dance-and-breast?ref=discovery
모인금액 33105
목표금액 30000
만든사람 Chithra Jeyaram
총 후원자수 309
/projects/730652731/foreign-puzzle-a-film-about-love-dance-and-breast/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0]
제작자지역 Austin, TX
장르 Documentary
서포트금액 [1, 10, 20, 25, 40, 50, 50, 75, 100, 150, 225, 250, 250, 500, 500, 750, 750, 2000, 7500, 300, 1000, 2500]
최대금액 7500
최소금액 1
서포트 단계 22
서포트 금액별 후원자수 ['10', '14', '8', '94', '27', '30', '13', '2', '19', '1', '2', '1', '1', '0', '1', '0', '0', '0', '1', '3', '3', '2']
최대 금액 후원자수 1
최소 금액 후원자수 10
링크 https://www.kickstarter.com/projects/katiewest/better-than-irl?ref=discovery
모인금

서포트 금액별 후원자수 ['268', '145', '49', '46', '32', '0', '16', '2', '2', '0', '20']
최대 금액 후원자수 0
최소 금액 후원자수 268
링크 https://www.kickstarter.com/projects/chaibio/open-qpcr-dna-diagnostics-for-everyone?ref=discovery
모인금액 202701
목표금액 50000
만든사람 Chai
총 후원자수 484
/projects/chaibio/open-qpcr-dna-diagnostics-for-everyone/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0]
제작자지역 Santa Clara, CA
장르 Hardware
서포트금액 [5, 29, 59, 140, 249, 1400, 1499, 1699, 2199, 10000, 1299]
최대금액 10000
최소금액 5
서포트 단계 11
서포트 금액별 후원자수 ['187', '83', '37', '17', '9', '2', '64', '5', '11', '1', '20']
최대 금액 후원자수 1
최소 금액 후원자수 187
링크 https://www.kickstarter.com/projects/435284672/niwa-the-worlds-first-smartphone-controlled-growin?ref=discovery
모인금액 151173
목표금액 100000
만든사람 Niwa
총 후원자수 565
/projects/435284672/niwa-the-worlds-first-smartphone-controlled-growin/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 

링크 https://www.kickstarter.com/projects/flyte/flyte-levitating-light?ref=discovery
모인금액 617258
목표금액 80000
만든사람 Flyte
총 후원자수 2,085
/projects/flyte/flyte-levitating-light/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5]
제작자지역 Stockholm, Sweden
장르 Product Design
서포트금액 [5, 149, 249, 1149, 2199, 4000, 199, 229, 239, 239, 479, 479, 529, 549]
최대금액 4000
최소금액 5
서포트 단계 14
서포트 금액별 후원자수 ['85', '239', '1,205', '17', '4', '0', '30', '30', '60', '60', '29', '250', '10', '10']
최대 금액 후원자수 0
최소 금액 후원자수 85
링크 https://www.kickstarter.com/projects/skysharks/sky-sharks?ref=discovery
모인금액 96377
목표금액 75000
만든사람 TEAM SKY SHARKS
총 후원자수 907
/projects/skysharks/sky-sharks/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0]
제작자지역 B

링크 https://www.kickstarter.com/projects/snapmaker/snapmaker-20-modular-3-in-1-3d-printers?ref=discovery
모인금액 7850866
목표금액 100000
만든사람 Snapmaker
총 후원자수 7,388
/projects/snapmaker/snapmaker-20-modular-3-in-1-3d-printers/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0, 0, 6, 6, 3, 44, 5, 3, 3, 3, 0, 2]
제작자지역 New Castle, DE
장르 3D Printing
서포트금액 [719, 719, 899, 899, 1079, 599, 719, 749, 899, 899, 899, 1079, 1079, 1079]
최대금액 1079
최소금액 599
서포트 단계 14
서포트 금액별 후원자수 ['199', '22', '599', '304', '1,078', '100', '150', '200', '300', '450', '200', '600', '1,200', '1,800']
최대 금액 후원자수 1,078
최소 금액 후원자수 100
링크 https://www.kickstarter.com/projects/andrewjames/street-fighting-man?ref=discovery
모인금액 8151
목표금액 6500
만든사람 Andrew James
총 후원자수 153
/projects/andrewjames/street-fighting-man/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8

링크 https://www.kickstarter.com/projects/elanlee/exploding-kittens?ref=discovery
모인금액 8782571
목표금액 10000
만든사람 Exploding Kittens
총 후원자수 219,382
/projects/elanlee/exploding-kittens/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0, 0, 6, 6, 3, 44, 5, 3, 3, 3, 0, 2, 3, 0, 2, 2, 30, 0, 8, 0, 0, 2, 3]
제작자지역 Los Angeles, CA
장르 Tabletop Games
서포트금액 [20, 35, 100, 500]
최대금액 500
최소금액 20
서포트 단계 4
서포트 금액별 후원자수 ['15,505', '202,934', '200', '5']
최대 금액 후원자수 5
최소 금액 후원자수 15,505
링크 https://www.kickstarter.com/projects/249675593/2nd-chess-and-science-festival-and-all-girls-chess?ref=discovery
모인금액 1654
목표금액 1000
만든사람 Anjelina Belakovskaia
총 후원자수 76
/projects/249675593/2nd-chess-and-science-festival-and-all-girls-chess/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 

서포트 금액별 후원자수 ['2,251', '26,238', '382', '4,671', '14,668', '2,062', '6,539', '1,791', '1,279', '2,822', '1,170', '3,141', '885', '147', '90', '88', '7', '30', '8', '8', '44', '44', '79', '12', '3', '6', '22', '2', '1', '1', '0', '0', '2', '13', '12', '4', '3', '5', '3']
최대 금액 후원자수 0
최소 금액 후원자수 2,251
링크 https://www.kickstarter.com/projects/167681950/the-animated-mind-of-oliver-sacks?ref=discovery
모인금액 124295
목표금액 100000
만든사람 Dempsey Rice
총 후원자수 1,288
/projects/167681950/the-animated-mind-of-oliver-sacks/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0, 0, 6, 6, 3, 44, 5, 3, 3, 3, 0, 2, 3, 0, 2, 2, 30, 0, 8, 0, 0, 2, 3, 6, 5, 0, 9, 5, 9, 0, 3, 6, 0, 0, 2]
제작자지역 Brooklyn, NY
장르 Animation
서포트금액 [5, 10, 15, 25, 30, 40, 45, 60, 75, 80, 85, 85, 100, 125, 135, 150, 160, 175, 300, 750, 750, 1000, 1250, 2000, 5000, 5000, 10000, 35, 400, 750, 750, 750, 

링크 https://www.kickstarter.com/projects/barisieurimmersion/barisieur-20-coffee-and-tea-brewing-alarm-clock?ref=discovery
모인금액 109558
목표금액 10000
만든사람 Barisieur Ltd.
총 후원자수 643
/projects/barisieurimmersion/barisieur-20-coffee-and-tea-brewing-alarm-clock/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0, 0, 6, 6, 3, 44, 5, 3, 3, 3, 0, 2, 3, 0, 2, 2, 30, 0, 8, 0, 0, 2, 3, 6, 5, 0, 9, 5, 9, 0, 3, 6, 0, 0, 2, 2, 0, 0, 0, 0, 30, 2, 3, 7, 2]
제작자지역 London, UK
장르 Product Design
서포트금액 [7, 19, 25, 29, 39, 48, 68, 69, 79, 239, 259, 319, 430, 199, 299]
최대금액 430
최소금액 7
서포트 단계 15
서포트 금액별 후원자수 ['21', '68', '7', '20', '90', '16', '13', '1', '5', '114', '47', '23', '20', '100', '50']
최대 금액 후원자수 20
최소 금액 후원자수 21
링크 https://www.kickstarter.com/projects/beehivebooks/i-will-live-forever-papercraft-comics-by-maelle-do?ref=discovery
모인금액 79044
목표금액 60000
만든사람 Beehive 

링크 https://www.kickstarter.com/projects/joue-play/joue-play-everyone-can-play-music?ref=discovery
모인금액 188399
목표금액 70000
만든사람 Joué
총 후원자수 613
/projects/joue-play/joue-play-everyone-can-play-music/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2, 11, 0, 3, 2, 2, 2, 5, 0, 8, 9, 3, 0, 0, 8, 0, 4, 2, 0, 0, 2, 5, 2, 0, 7, 0, 3, 4, 2, 6, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 2, 6, 5, 10, 2, 19, 4, 0, 5, 0, 0, 6, 6, 3, 44, 5, 3, 3, 3, 0, 2, 3, 0, 2, 2, 30, 0, 8, 0, 0, 2, 3, 6, 5, 0, 9, 5, 9, 0, 3, 6, 0, 0, 2, 2, 0, 0, 0, 0, 30, 2, 3, 7, 2, 11, 2, 0, 0, 0, 4, 0, 4, 2]
제작자지역 Bordeaux, France
장르 Hardware
서포트금액 [5, 267, 267, 299, 460]
최대금액 460
최소금액 5
서포트 단계 5
서포트 금액별 후원자수 ['17', '145', '108', '268', '56']
최대 금액 후원자수 56
최소 금액 후원자수 17
링크 https://www.kickstarter.com/projects/664592345/the-movie-that-will-change-the-way-you-look-at-she-0?ref=discovery
모인금액 12130
목표금액 11111
만든사람 Yuri Chicovsky
총 후원자수 81
/projects/664592345/the-movie-that-will-change-the-way-you-look-at-she-0/creator_bio
개설횟수 [25, 3, 0, 2, 6, 0, 2

In [7]:
success_list = []
result = []

for money_list,pledged_money_list,creator_list,backers_list,final_created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list in zip(money_list,pledged_money_list,creator_list,backers_list,final_created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list):       
    success_list = [money_list,pledged_money_list,creator_list,backers_list,final_created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list]
    result.append(success_list)

In [8]:
import pandas as pd
df = pd.DataFrame(result, columns = ['money','pledged_money','creator','backers','Number of projects opened','Producer area','genre','Support amount list','The highest amount of support','The minimum amount of support','Total number of support steps','List of supporters','Maximum amount of sponsors','Minimum amount of sponsors'])
df.to_csv('success_link_feature.csv',encoding = 'utf-8-sig')

In [ ]:
df